In [1]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
#from sklearn.model_selection import train_test_split
import tensorflow as tf
import librosa
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
import numpy as np
from pydub import AudioSegment
from pydub.playback import play
from scipy.io import wavfile
from tempfile import mktemp

from datasets import load_dataset

2023-11-28 15:49:12.847489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TIME_STEPS = 2
LETTER_CONVERSION = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q','r','s','t','u','v','w','x','y','z',' ','.', '' ]
image_name = 0

x_processed_data = []
y_processed_data = []

In [3]:
ds = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='train[:100]')

Found cached dataset common_voice_11_0 (/home/hetricke/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)


In [4]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms

In [5]:
def convertToNum(s):
    try: 
        return LETTER_CONVERSION.index(s)
    except:
        return -1

In [6]:
def convertToLetter(val):
    return LETTER_CONVERSION[val]

In [7]:
def transcript_prep(transcription, audio_len):

    global max_transcript_size     

    if transcription.startswith('"') and transcription.endswith('"'):
        # we can remove trailing quotation marks as they do not affect the transcription
        transcription = transcription[1:-1]

    transcription = transcription.lower()
    
    if len(transcription) > 0:
        if transcription[-1] not in [".", "?", "!"]:
            # append a full-stop to sentences that do not end in punctuation
            transcription = transcription + "."

    transcript_pieces = list(transcription)

    for i in range(len(transcript_pieces)):
        transcript_pieces[i] = convertToNum(transcript_pieces[i])

    return transcript_pieces


In [10]:
def prepare_dataset(batch):

    global image_name
    global x_processed_data
    global y_processed_data

    #reads in mp3
    mp3_audio = AudioSegment.from_file(batch['path'], format="mp3")  # read mp3

    if(round(mp3_audio.duration_seconds) == 0):
        return batch

    #removes silent audio from the beginning and end
    start_trim = detect_leading_silence(mp3_audio)
    end_trim = detect_leading_silence(mp3_audio.reverse())
    duration = len(mp3_audio)    
    trimmed_sound = mp3_audio[start_trim:duration-end_trim]

    #converts the mp3 into a wav file
    wname = mktemp('.wav')  # use temporary file
    trimmed_sound.export(wname, format="wav")  # convert to wav
    FS, audio_data = wavfile.read(wname)  # read wav file


    #creates a file name for the spectrogram
    file_name = "images/"+ str(image_name) + ".png"
    image_name = image_name + 1

    #creates and saves the spectrogram
    plt.figure()
    plt.specgram(audio_data, Fs=FS, NFFT=128, noverlap=0)  # plot
    plt.axis('off')
    plt.savefig(file_name, bbox_inches='tight')

    #clears the figure for the next audio transcript- otherwise it just overwrites the image, which causes Problems
    plt.close()


    #loads the spectrogram and turns it into an array
    img = keras.preprocessing.image.load_img(file_name)
    img_array = keras.preprocessing.image.img_to_array(img)


    #slices the image array into appriopriate sizes for each chunk
    samples = len(batch["sentence"])
    features = int(img_array.shape[0]/len(batch["sentence"]))

    divided_img_array = []

    index = 0
    x_data = []


    for i in range(samples):
        feature_array = img_array[index:index+features]
        new_dim = feature_array.shape[0]*feature_array.shape[1]
        feature_array = feature_array.reshape(new_dim, -1)

        original_length = feature_array.shape[0]
        padding = 66908-original_length
        feature_array = np.pad(feature_array, [(0,padding), (0,0)], mode='constant')

        feature_array = feature_array.tolist()
        divided_img_array.append(feature_array)

        x_data.append([])
        datapoint = divided_img_array
        x_data[i] = datapoint[::-1]

        
        index = index + features

    for i in range(samples):
        x_data.append([])
        datapoint = divided_img_array[:i]
        x_data[i] = datapoint[::-1]

    x_processed_data.append(x_data)

    y_processed_data.append(transcript_prep(batch["sentence"], samples))
    


    #reshape the spectogram into a 3d array that goes (num samples x timesteps x features)
    #1 sample is how many times this gets run through
    #timesteps is how far backward to grab (so that's a value that can be played with!)
    #num samples is I think how long it is (so grab the mp3 length)
    #slice the image accordingly
    #convert the image into an array (reference the obj detection) to use for the features
    #profit

    #so the min time is three, which, at 515 timesteps, gets 171.6666 columns of data per second
    

    return batch


In [11]:
image_name = 0
#ds = ds.map(prepare_dataset, desc="preprocess dataset")

count = 0
for data in ds:
    print(count)
    prepare_dataset(data)
    count = count + 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


/home/hetricke/anaconda3/envs/speech_recog_env/lib/python3.11/site-packages/matplotlib/axes/_axes.py:7939: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


30
31
32


In [46]:
# split = ds.train_test_split(test_size = 0.2,train_size = 0.8)
# train = split['train']
# test = split['test']

x_train = x_processed_data[:80]
y_train = y_processed_data[:80]

x_test = x_processed_data[:-20]
y_test = y_processed_data[:-20]


In [35]:
#build model
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(units = 128, activation = 'relu', input_shape = (66908, 3)))
model.add(keras.layers.Dense(units = 128, activation = 'relu'))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation ='relu'))
model.add(keras.layers.Dense(len(LETTER_CONVERSION), activation = 'softmax'))
model.compile(optimizer='adam', metrics=['accuracy'])

In [ ]:
print(x_train[0])
print(y_train[0])

In [ ]:
#train model
#what you might have to do is iteratively run fit on all the audio samples

model.fit(x_train[0], y_train[0], epochs=10, batch_size=1)


In [ ]:
#add computer i/o for speech recognition